In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gdown

In [ ]:
import gdown,zipfile

# Google Drive file ID from the provided URL
file_id = "131VnscdKVOcv6AyDCv4zrTWApHyeuZ4d"
# Destination directory for extracted data
output_directory = "/kaggle/working/DATA"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Define the zip file name
zip_filename = os.path.join(output_directory, "train.zip")

# Download the data from Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_filename, quiet=False)

# Check if the downloaded file is a ZIP archive
if zipfile.is_zipfile(zip_filename):
    # Extract the data from the zip file
    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        zip_ref.extractall(output_directory)
    print("Data downloaded and extracted successfully.")
else:
    print("The downloaded file is not a valid ZIP archive.")

In [ ]:
import gdown,zipfile

# Google Drive file ID from the provided URL
file_id = "1isg6zgjE3SlwgRYfYn4AvLud6MQj3C_b"
# Destination directory for extracted data
output_directory = "/kaggle/working/DATA/images"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Define the zip file name
zip_filename = os.path.join(output_directory, "images.zip")

# Download the data from Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_filename, quiet=False)

# Check if the downloaded file is a ZIP archive
if zipfile.is_zipfile(zip_filename):
    # Extract the data from the zip file
    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        zip_ref.extractall(output_directory)
    print("Data downloaded and extracted successfully.")
else:
    print("The downloaded file is not a valid ZIP archive.")

In [ ]:
from IPython.display import clear_output
# Install dew required libraries
!pip install pyunpack
!pip install patool
!pip install rasterio
clear_output()
# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import rasterio
import cv2

In [ ]:
# Load files
path_tabular_data= "/kaggle/input/tabdata/"

df_train = gpd.read_file(f"{path_tabular_data}train.shp")
df_test = gpd.read_file(f"{path_tabular_data}test.shp")
samplesubmission = pd.read_csv(f"{path_tabular_data}SampleSubmission.csv")

# Preview train dataset
df_train.head()

In [ ]:
df_train.nunique()

In [ ]:
df_test.head()

In [ ]:
samplesubmission.head()

In [ ]:
df_train.shape, df_test.shape, samplesubmission.shape

In [ ]:
sns.displot(df_train, x="target")

In [ ]:
path_satellite__data= "/kaggle/working/DATA/images/GeoAIHack2023/"
images_list= os.listdir(path_satellite__data)

In [ ]:
# Open 1 timestamp example image
image_path = f"{path_satellite__data}{images_list[23]}"
with rasterio.open(image_path) as src:
    # Get the image metadata
    metadata = src.meta
    # Read the image as an array
    image = src.read()

In [ ]:
# Plot the Spectral-Bands of the image
# The Sentinel-2 Spectral band includes "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"
fig, ax = plt.subplots(2, 5, figsize=(25, 10))

for i in range(2):
    for j in range(5):
        ax[i,j].imshow(image[i*4+j,:,:])
        ax[i,j].set_title("Band {}".format(i*4+j+1))
        ax[i,j].axis("off")

## Field Visualization to see if there's same distrubtion of the bands over Timestamps

# visualization of Olive Fileds with different Timestamps

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of timestamp names
timestamp_names = ["Timestamp {}".format(i) for i in range(24)]


for t in range(24):  # loop over timestamps
    fig, ax = plt.subplots(4, 7, figsize=(25, 10))

    for i in range(4):
        for j in range(7):
            ax[i, j].imshow(new_data[0, t, i * 6 +j , :, :])
            ax[i, j].set_title("Band {}".format(i * 6 + j + 1))
            ax[i, j].axis("off")
    
    fig.suptitle(timestamp_names[t], fontsize=16)  # Add timestamp name as the overall title
    plt.show()  # show each timestamp in a separate figure


# visualization of Not_Olive Fileds with different Timestamps

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of timestamp names
timestamp_names = ["Timestamp {}".format(i) for i in range(24)]


for t in range(24):  # loop over timestamps
    fig, ax = plt.subplots(4, 7, figsize=(25, 10))

    for i in range(4):
        for j in range(7):
            ax[i, j].imshow(new_data[1, t, i * 6 +j , :, :])
            ax[i, j].set_title("Band {}".format(i * 6 + j + 1))
            ax[i, j].axis("off")
    
    fig.suptitle(timestamp_names[t], fontsize=16)  # Add timestamp name as the overall title
    plt.show()  # show each timestamp in a separate figure


# ****Machine Learning Approach (withe Vegetation Indices,Geometry Features and Statistical Features with all Timestamps)****

In [ ]:
df = pd.DataFrame()
for timestamp in range(24):  # Assuming timestamps range from 0 to 24
    image_path = f"{path_satellite__data}{images_list[timestamp]}"
    
    with rasterio.open(image_path) as src:
        # Get the image metadata
        metadata = src.meta
        # Read the image as an array
        image = src.read()
    # Concatenate the training and testing dataframes into a single dataframe
    df_all= pd.concat([df_train, df_test], ignore_index=True)

    # Initialize an empty list to store the processed data
    processed_data= []
    # Loop through each field in the concatenated dataframe
    for index, row in df_all.iterrows():
        # Get the geometry of the field
        field_geometry = row['geometry']
        # Create a mask for the field
        field_mask = geometry_mask([field_geometry], out_shape=src.shape, transform=src.transform, invert=True)
        # Extract the image data for the field
        field_data = image[:, field_mask]

        # Compute the mean and standard deviation of each channel of the image data
        mean_values = np.mean(field_data, axis=1)
        std_values = np.std(field_data, axis=1)
        max_values = np.max(field_data, axis=1)
        min_values = np.min(field_data, axis=1)
        median_values=np.median(field_data, axis=1)

        # Concatenate the mean and standard deviation values into a single feature vector
        features = np.concatenate([mean_values, std_values,max_values,min_values,median_values])
        # Append the feature vector to the list of processed data
        processed_data.append(features)

    # Convert the list of processed data into a pandas dataframe
    processed_data_df= pd.DataFrame(processed_data, columns=[f"mean_channel_{i}" for i in range(0, 10)] + [f"std_channel_{i}" for i in range(0, 10)]+[f"max_channel_{i}" for i in range(0, 10)] + [f"min_channel_{i}" for i in range(0, 10)]+[f"median_channel_{i}" for i in range(0, 10)])
    data=pd.concat(processed_data_df,axis=1)
data["ID"]= df_all["ID"]    

In [ ]:
import pandas as pd
import numpy as np
import rasterio
from rasterio.features import geometry_mask

# Initialize an empty dataframe to store the processed data
data = pd.DataFrame()
df_all= pd.concat([df_train, df_test], ignore_index=True)

# Assuming timestamps range from 0 to 24
for timestamp in range(24):
    image_path = f"{path_satellite__data}{images_list[timestamp]}"
    
    with rasterio.open(image_path) as src:
        # Get the image metadata
        metadata = src.meta
        # Read the image as an array
        image = src.read()

    # Initialize an empty list to store the processed data for the current timestamp
    processed_data = []
    
    # Loop through each field in the concatenated dataframe
    for index, row in df_all.iterrows():
        # Get the geometry of the field
        field_geometry = row['geometry']
        # Create a mask for the field
        field_mask = geometry_mask([field_geometry], out_shape=src.shape, transform=src.transform, invert=True)
        # Extract the image data for the field
        field_data = image[:, field_mask]

        # Compute the mean and standard deviation of each channel of the image data
        mean_values = np.mean(field_data, axis=1)
        std_values = np.std(field_data, axis=1)
        max_values = np.max(field_data, axis=1)
        min_values = np.min(field_data, axis=1)
        median_values = np.median(field_data, axis=1)

        # Concatenate the mean and standard deviation values into a single feature vector
        features = np.concatenate([mean_values, std_values, max_values, min_values, median_values])
        # Append the feature vector to the list of processed data for the current timestamp
        processed_data.append(features)

    # Convert the list of processed data into a pandas dataframe for the current timestamp
    processed_data_df = pd.DataFrame(processed_data, columns=[f"mean_channel_{i}_timestamp_{timestamp}" for i in range(0, 10)] + [f"std_channel_{i}_timestamp_{timestamp}" for i in range(0, 10)]+[f"max_channel_{i}_timestamp_{timestamp}" for i in range(0, 10)] + [f"min_channel_{i}_timestamp_{timestamp}" for i in range(0, 10)]+[f"median_channel_{i}_timestamp_{timestamp}" for i in range(0, 10)])
    if timestamp<1:
        processed_data_df["ID"] = df_all["ID"]
    else:
        pass
    # Concatenate the current timestamp's processed data to the main dataframe
    data = pd.concat([data, processed_data_df], axis=1)

# You may want to reset the index of the resulting dataframe
data.shape

# **Feature Engineering:**
# 1. Geometry Features
# 2. Vegetation Indices
# 3. Statistics on all Timestamps

In [ ]:
def calculate_polygon_features(polygon):
    area = polygon.area
    perimeter = polygon.length
    centroid = polygon.centroid
    bounding_box = polygon.bounds
    aspect_ratio = bounding_box[2] - bounding_box[0] / (bounding_box[3] - bounding_box[1])
    compactness = 4 * np.pi * area / perimeter**2
    convex_hull = polygon.convex_hull
    orientation = np.arctan2(centroid.y - bounding_box[1], centroid.x - bounding_box[0])
    perimeter_to_area_ratio = perimeter / area
    circumradius = np.sqrt((bounding_box[2] - bounding_box[0])**2 + (bounding_box[3] - bounding_box[1])**2) / 2
    inscribed_circle_radius = area / perimeter * 2 / np.pi

    return {
        'Area': area,
        'Perimeter': perimeter,
        'Aspect Ratio': aspect_ratio,
        'Compactness': compactness,
        'Orientation': orientation,
        'Perimeter-to-Area Ratio': perimeter_to_area_ratio,
        'Circumradius': circumradius,
        'Inscribed Circle Radius': inscribed_circle_radius
    }

data["geometry"]= df_all["geometry"]    
# Apply the function to the GeoDataFrame
data['Polygon_Features'] = data['geometry'].apply(calculate_polygon_features)

# Create new columns for each feature
data['Area'] = data['Polygon_Features'].apply(lambda x: x['Area'])
data['Perimeter'] = data['Polygon_Features'].apply(lambda x: x['Perimeter'])
data['Aspect_Ratio'] = data['Polygon_Features'].apply(lambda x: x['Aspect Ratio'])
data['Compactness'] = data['Polygon_Features'].apply(lambda x: x['Compactness'])
data['Orientation'] = data['Polygon_Features'].apply(lambda x: x['Orientation'])
data['Perimeter_to_Area_Ratio'] = data['Polygon_Features'].apply(lambda x: x['Perimeter-to-Area Ratio'])
data['Circumradius'] = data['Polygon_Features'].apply(lambda x: x['Circumradius'])
data['Inscribed_Circle_Radius'] = data['Polygon_Features'].apply(lambda x: x['Inscribed Circle Radius'])

# Drop the original Polygon_Features column if needed
data = data.drop(columns=['Polygon_Features'])
data = data.drop(columns=["geometry"])




In [ ]:
def calculate_vegetation_indices(b2, b3, b4, b5, b6, b7, b8, b8A, b11, b12):
    indices = {}

    # Common indices
    indices['NDVI'] = (b8 - b4) / (b8 + b4)
    indices['EVI'] = 2.5 * (b8 - b4) / (b8 + 6 * b4 - 7.5 * b2 + 1)
    indices['SAVI'] = (1 + 0.5) * (b8 - b4) / (b8 + b4 + 0.5)
    indices['MNDWI'] = (b3 - b11) / (b3 + b11)
    indices['NDII'] = (b8 - b11) / (b8 + b11)
    indices['NBR'] = (b8 - b12) / (b8 + b12)

    # Additional indices
    indices['GNDVI'] = (b8 - b3) / (b8 + b3)
    indices['SR'] = b8 / b4
    indices['VARI'] = (b3 - b2) / (b3 + b2 - b11)
    indices['LAI'] = 1/6 * np.log(6 * (b4 - b3) / (b4 + b3))
    indices['PSRI'] = (b4 - b3) / b6
    indices['NLI'] = (b8**2 - b4) / (b8**2 + b4)

    # Additional indices continued
    indices['WSI'] = (b3 - b5) / (b3 + b5)
    indices['LCI'] = (b8A - b7) / (b8A + b7)
    indices['CCCI'] = (b8 - b4) / (b8 + b4)
    indices['RENDVI'] = (b8 - b5) / (b8 + b5)
    indices['TGI'] = -0.5 * (190 * (b2 - b3) - 120 * (b2 - b4))
    indices['NPQI'] = (b4 - b2) / (b4 + b2)
    indices['CRI'] = (b7 - b5) / (b7 + b5)
    indices['NDRE'] = (b5 - b4) / (b5 + b4)
    
    indices['MTVI'] = 1.5 * (1.2 * (b8 - b6) - 2.5 * (b4 - b3))
    indices['EVI2'] = 2.5 * (b8 - b2) / (b8 + 2.4 * b2 + 1)
    indices['RENDAI'] = (b7 - b5) / (b7 + b5)
    indices['MTCI'] = (b7 - b6) / (b6 - b5)
    indices['VARI'] = (b3 - b2) / (b3 + b2 + b11)
    indices['LWI'] = (b8A - b11) / (b8A + b11)
    indices['TSAVI'] = 0.8 * (b8 - b4) / ((b8 + b4 + 0.6) * (1 + 0.6))
    indices['CIRED2'] = (b8 - b5) / (b7 - b5)
    indices['VGI'] = b3 / (b2 + b4)
    indices['NDWI2'] = (b11 - b12) / (b11 + b12)


    return indices



In [ ]:
list_stat = ["mean", "median", "max", "min", "std"]
channels = range(10)  # Assuming channels are from 0 to 9

vegetation_indices = ["NDVI","EVI","SAVI","MNDWI","NDII","NBR","GNDVI","SR","VARI","LAI","PSRI","NLI","WSI","LCI","CCCI","RENDVI","TGI","NPQI","CRI","NDRE","MTVI","EVI2","RENDAI","MTCI","VARI","LWI","TSAVI","CIRED2","VGI","NDWI2"]

for timestamp in range(24):
    for stat in list_stat:
        for index_name in vegetation_indices:
            bands = {
                'b2': data[f'{stat}_channel_0_timestamp_{timestamp}'],
                'b3': data[f'{stat}_channel_1_timestamp_{timestamp}'],
                'b4': data[f'{stat}_channel_2_timestamp_{timestamp}'],
                'b5': data[f'{stat}_channel_3_timestamp_{timestamp}'],
                'b6': data[f'{stat}_channel_4_timestamp_{timestamp}'],
                'b7': data[f'{stat}_channel_5_timestamp_{timestamp}'],
                'b8': data[f'{stat}_channel_6_timestamp_{timestamp}'],
                'b8A': data[f'{stat}_channel_7_timestamp_{timestamp}'],
                'b11': data[f'{stat}_channel_8_timestamp_{timestamp}'],
                'b12': data[f'{stat}_channel_9_timestamp_{timestamp}'],
            }
            index_values = calculate_vegetation_indices(**bands)[index_name]

            # Add the new index columns to the dataframe
            data[f'{index_name}_{stat}_timestamp_{timestamp}'] = index_values

            

    
            # Add more conditions for other vegetation indices
            
            # Add the new index columns to the dataframe
            data[f'{index_name}_{stat}_timestamp_{timestamp}'] = index_values


In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.to_csv("final_data.csv", index=False)

In [ ]:
# Retrive the train and test sets
from sklearn.model_selection import train_test_split

processed_train_df= data.iloc[:len(df_train), :]
processed_test_df= data.iloc[len(df_train):, :]
processed_test_df.reset_index(drop=True, inplace=True)

# Split the dataset into training and testing sets

X = processed_train_df.drop(columns=["ID"])
y = df_train["target"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **Testing With RF**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create an instance of the RandomForestClassifier class
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
X_train=X_train.fillna(0)
X_test=X_test.fillna(0)
X_train.replace([np.inf, -np.inf], 0, inplace=True)
X_test.replace([np.inf, -np.inf], 0, inplace=True)
# Fit the model to the training data
rfc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rfc.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Applying Yeo-Johnson transformation with Catboost

In [ ]:
from catboost import CatBoostClassifier, Pool, cv, CatBoost
from sklearn.preprocessing import PowerTransformer
X.replace([np.inf, -np.inf],0, inplace=True)
#X_test.replace([np.inf, -np.inf], 0, inplace=True)

#power_transformer = PowerTransformer(method='yeo-johnson')
#X_train_transformed = power_transformer.fit_transform(X_train)
#X_test_transformed = power_transformer.transform(X_test)

# Create a CatBoost pool for training data
train_pool = Pool(X, y)

# Define CatBoost parameters


# Create and train CatBoost classifier
catboost_clf = CatBoostClassifier()
catboost_clf.fit(train_pool, eval_set=(X_test, y_test), plot=True)


In [ ]:
from sklearn.metrics import *
from sklearn.model_selection import *
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Testing Lgbm

In [ ]:
# Fit the model
model = lgb.LGBMClassifier()
model.fit(X, y)
y_pred = model.predict(X_test)

# Score with RMSE
print('Score:', accuracy_score(y_test, y_pred))

# Hyperparamater Tuning For Lgbm with Optuna

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def objective(trial):
    # Split the data into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define hyperparameters to optimize
    params = {
        'objective': 'binary',
        'metric': 'binary_error',  # You can change the metric based on your preference
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # Create a LightGBM model
    model = lgb.LGBMClassifier(**params)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = model.predict(X_valid)

    # Calculate accuracy for the validation set
    accuracy = accuracy_score(y_valid, y_pred)

    return 1.0 - accuracy  # Optuna minimizes the objective function, so we use 1.0 - accuracy

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Get the best hyperparameters
best_params = study.best_params
print('Best Hyperparameters:', best_params)

# Train the final model with the best hyperparameters on the full dataset
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X, y)

# Make predictions on the test set
y_pred_test = final_model.predict(X_test)

# Evaluate the final model on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)
print('Test Accuracy:', test_accuracy)


# Feature Importance

In [ ]:
feature_importance = model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)[:10]

# Display the feature importance
print('\nFeature Importance:')
print(feature_importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.bar(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')


# Submission

In [ ]:
test=processed_test_df.drop(columns=["ID"])
#test = power_transformer.fit_transform(test)

# Generate the test set predictions

test_preds = model.predict(test)

# Create a submission file using the predictions of the Random Forest model
lgb_submission= samplesubmission.copy()
lgb_submission["target"]= test_preds

In [ ]:
lgb_submission.to_csv("final_sub_lgb_allX.csv", index=False)

In [ ]:
lgb_submission["target"].value_counts()